# 第1章：基础ReActAgent实现

## 本章目标

本章将实现最基础的ReAct Agent，帮助你理解LangGraph的核心概念：

- **StateGraph**: 状态图，定义Agent的执行流程
- **MessagesState**: 消息状态，管理对话历史
- **Node**: 节点，执行具体的逻辑
- **Edge**: 边，连接节点定义流程
- **ToolNode**: 工具节点，执行工具调用
- **Conditional Edges**: 条件边，根据状态动态路由

## 核心概念

ReAct (Reasoning + Acting) 是一种让LLM进行推理并执行动作的模式：
1. **Reasoning**: LLM分析问题，决定是否需要使用工具
2. **Acting**: 如果需要，调用工具获取信息
3. **Loop**: 重复上述过程直到得到最终答案


## 💡 这是 Claude Code 的什么功能？

ReAct 循环是 Claude Code 能够**自主完成编程任务**的核心机制。它让 AI 能够：
- 分析问题 → 决定使用哪些工具 → 执行工具 → 根据结果继续思考 → 循环

**典型场景**：当你说"创建一个 FastAPI 项目"，Claude Code 会自动：
1. 思考需要哪些文件 → 调用 Write 工具创建 main.py
2. 思考需要哪些依赖 → 调用 Write 工具创建 requirements.txt  
3. 思考是否需要安装 → 调用 Bash 工具执行 pip install
4. 完成后告诉你结果

这个"思考-行动-再思考"的循环，就是本章要实现的核心。

## 1. 环境设置

In [ ]:
# 导入必要的库
import os
from typing import Literal
from IPython.display import Image, display

# LangGraph核心
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition, create_react_agent

# LangChain
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage

# LLM - 可选择OpenAI或通义千问
# from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatTongyi  # 使用通义千问时取消注释

In [ ]:
# 配置环境变量
# os.environ["OPENAI_API_KEY"] = "your-api-key"  # 如果需要

# 配置LangSmith追踪（可选，用于调试）
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "chapter-01-basic-react-agent"

In [ ]:
# 初始化LLM
# 选项1: OpenAI
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 选项2: 通义千问 (取消注释使用)
llm = ChatTongyi(model="qwen3-max", temperature=0)

print(f"✅ LLM初始化成功: {llm.model_name}")

## 2. 定义工具

我们定义三个简单的数学工具来演示Agent如何调用工具。使用 `@tool` 装饰器可以轻松将Python函数转换为LangChain工具。

In [ ]:
@tool
def multiply(a: int, b: int) -> int:
    """两数相乘
    
    Args:
        a: 第一个整数
        b: 第二个整数
        
    Returns:
        两数的乘积
    """
    return a * b


@tool
def add(a: int, b: int) -> int:
    """两数相加
    
    Args:
        a: 第一个整数
        b: 第二个整数
        
    Returns:
        两数的和
    """
    return a + b


@tool
def divide(a: int, b: int) -> float:
    """两数相除
    
    Args:
        a: 被除数
        b: 除数
        
    Returns:
        两数的商
    """
    if b == 0:
        return "错误：除数不能为0"
    return a / b


# 工具列表
tools = [add, multiply, divide]

print(f"✅ 定义了 {len(tools)} 个工具: {[t.name for t in tools]}")

In [ ]:
# 查看工具的详细信息
for tool_obj in tools:
    print(f"\n工具名: {tool_obj.name}")
    print(f"描述: {tool_obj.description}")
    print(f"参数: {tool_obj.args}")

## 3. 方式一：手动构建Agent

首先，我们手动构建一个ReAct Agent，这样可以深入理解每个组件的作用。

### 3.1 定义节点函数

In [ ]:
# 将工具绑定到LLM
llm_with_tools = llm.bind_tools(tools)

# 创建工具名称到工具对象的映射
tools_by_name = {tool.name: tool for tool in tools}

print("✅ LLM已绑定工具")
print(f"可用工具: {list(tools_by_name.keys())}")

In [ ]:
def llm_call(state: MessagesState):
    """LLM节点：决定是否需要调用工具
    
    这个节点会调用LLM，LLM会分析用户问题并决定：
    1. 直接回答（如果不需要工具）
    2. 调用工具（如果需要额外信息）
    """
    # 添加系统提示
    messages = [
        SystemMessage(content="你是一个有帮助的AI助手，擅长进行数学计算。")
    ] + state["messages"]
    
    # 调用LLM
    response = llm_with_tools.invoke(messages)
    
    # 返回更新后的消息列表
    return {"messages": [response]}


def tool_node(state: MessagesState):
    """工具节点：执行工具调用
    
    这个节点会：
    1. 从最后一条AI消息中提取工具调用
    2. 执行对应的工具
    3. 返回工具执行结果
    """
    results = []
    
    # 获取最后一条消息中的工具调用
    last_message = state["messages"][-1]
    
    # 遍历所有工具调用
    for tool_call in last_message.tool_calls:
        # 获取工具
        tool = tools_by_name[tool_call["name"]]
        
        # 执行工具
        observation = tool.invoke(tool_call["args"])
        
        # 创建工具消息
        results.append(
            ToolMessage(
                content=str(observation),
                tool_call_id=tool_call["id"]
            )
        )
    
    return {"messages": results}


print("✅ 节点函数定义完成")

### 3.2 定义条件判断函数

条件判断函数决定了Agent的下一步行为：
- 如果LLM返回了工具调用 → 前往工具节点
- 如果LLM直接回答 → 结束

In [ ]:
def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    """判断是否需要继续执行工具
    
    Returns:
        "tools": 如果需要执行工具
        END: 如果可以结束对话
    """
    messages = state["messages"]
    last_message = messages[-1]
    
    # 检查最后一条消息是否包含工具调用
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "tools"
    
    # 否则结束
    return END


print("✅ 条件判断函数定义完成")

### 3.3 构建状态图

In [ ]:
# 创建状态图构建器
agent_builder = StateGraph(MessagesState)

# 添加节点
agent_builder.add_node("llm_call", llm_call)
agent_builder.add_node("tools", tool_node)

# 添加边
agent_builder.add_edge(START, "llm_call")  # 从START开始，进入llm_call节点

# 添加条件边：从llm_call出来后，根据should_continue的结果决定下一步
agent_builder.add_conditional_edges(
    "llm_call",
    should_continue,
    {
        "tools": "tools",  # 如果需要工具，前往tools节点
        END: END,          # 否则结束
    }
)

# 添加从tools回到llm_call的边（形成循环）
agent_builder.add_edge("tools", "llm_call")

# 编译图
agent = agent_builder.compile()

print("✅ 状态图构建完成")

### 3.4 可视化图结构

In [ ]:
# 显示图的结构
try:
    display(Image(agent.get_graph(xray=True).draw_mermaid_png()))
except Exception as e:
    print(f"无法显示图像（需要安装graphviz）: {e}")
    print("\n图的文本表示:")
    print(agent.get_graph().to_mermaid())

### 3.5 测试手动构建的Agent

In [ ]:
# 测试1: 简单加法
print("="*50)
print("测试1: 七加八等于几？")
print("="*50)

messages = [HumanMessage(content="七加八等于几？")]
result = agent.invoke({"messages": messages})

# 打印所有消息
for msg in result["messages"]:
    msg.pretty_print()

In [ ]:
# 测试2: 复杂计算
print("="*50)
print("测试2: 计算 (5 + 3) * 4 等于多少？")
print("="*50)

messages = [HumanMessage(content="计算 (5 + 3) * 4 等于多少？")]
result = agent.invoke({"messages": messages})

for msg in result["messages"]:
    msg.pretty_print()

In [ ]:
# 测试3: 不需要工具的问题
print("="*50)
print("测试3: 你好！请介绍一下你自己")
print("="*50)

messages = [HumanMessage(content="你好！请介绍一下你自己")]
result = agent.invoke({"messages": messages})

for msg in result["messages"]:
    msg.pretty_print()

## 4. 方式二：使用预构建Agent

LangGraph提供了 `create_react_agent` 函数，可以快速创建ReAct Agent，省去手动构建的步骤。

### 对比

| 方式 | 优点 | 缺点 |
|------|------|------|
| 手动构建 | 完全控制，深入理解 | 代码较多 |
| 预构建 | 代码简洁，快速开发 | 自定义程度较低 |

In [ ]:
# 使用预构建的create_react_agent
prebuilt_agent = create_react_agent(
    llm,
    tools=tools,
    prompt="你是一个有帮助的AI助手，擅长进行数学计算。"
)

print("✅ 预构建Agent创建完成")

In [ ]:
# 可视化预构建Agent的图结构
try:
    display(Image(prebuilt_agent.get_graph().draw_mermaid_png()))
except Exception as e:
    print(f"无法显示图像: {e}")
    print("\n图的文本表示:")
    print(prebuilt_agent.get_graph().to_mermaid())

In [ ]:
# 测试预构建Agent
print("="*50)
print("使用预构建Agent: 计算 12 除以 3")
print("="*50)

messages = [HumanMessage(content="计算 12 除以 3")]
result = prebuilt_agent.invoke({"messages": messages})

for msg in result["messages"]:
    msg.pretty_print()

## 5. 深入理解：MessagesState

`MessagesState` 是LangGraph中用于管理对话历史的特殊状态类型。

In [ ]:
# 查看MessagesState的定义
from langgraph.graph.message import add_messages

# MessagesState等价于以下TypedDict:
# class MessagesState(TypedDict):
#     messages: Annotated[list[BaseMessage], add_messages]

print("MessagesState关键特性:")
print("1. messages字段使用add_messages reducer")
print("2. 新消息会自动追加到列表末尾")
print("3. 保持完整的对话历史")

In [ ]:
# 演示消息累积
print("演示多轮对话:")
print("="*50)

# 第一轮
messages = [HumanMessage(content="5加3等于多少？")]
result = prebuilt_agent.invoke({"messages": messages})

print(f"\n第一轮后，消息数量: {len(result['messages'])}")

# 第二轮 - 继续使用之前的消息
result["messages"].append(HumanMessage(content="那再乘以2呢？"))
result = prebuilt_agent.invoke({"messages": result["messages"]})

print(f"第二轮后，消息数量: {len(result['messages'])}")

# 打印完整历史
print("\n完整对话历史:")
for i, msg in enumerate(result["messages"]):
    print(f"\n[{i+1}] {msg.__class__.__name__}")
    print(f"内容: {msg.content[:100] if msg.content else '(工具调用)'}...")

## 6. 核心概念总结

### StateGraph执行流程

```
START
  ↓
llm_call节点
  ↓
判断条件 (should_continue)
  ├─→ 有工具调用? → tools节点 → 返回llm_call (循环)
  └─→ 无工具调用? → END
```

### 关键组件

1. **StateGraph**: 图的容器，管理所有节点和边
2. **MessagesState**: 状态定义，包含messages字段
3. **Node函数**: 接收state，返回state更新
4. **Edge**: 定义节点间的转换
5. **Conditional Edge**: 根据条件动态选择下一个节点


## 7. 练习题

### 练习1: 添加新工具
尝试添加一个 `power(a, b)` 工具，计算a的b次方。

### 练习2: 自定义System Prompt
修改系统提示词，让Agent在回答时更有个性。

### 练习3: 添加日志
在节点函数中添加print语句，观察Agent的执行流程。

### 思考题
1. 为什么需要从tools节点返回到llm_call节点？
2. 如果不使用条件边，直接用普通边会发生什么？
3. MessagesState的add_messages reducer有什么好处？

## 8. 下一步

恭喜！你已经掌握了LangGraph的基础知识。

在下一章（第2章），我们将学习：
- 如何实现人在环路（Human-in-the-Loop）
- 使用 `interrupt()` 暂停执行
- 使用 `Command` 动态路由
- 使用 `checkpointer` 持久化状态